In [1]:
#!/usr/bin/env python3
"""
Stimulus-held-out cross-validated encoding:
ViT PCs  -->  Neural PCs  (ridge regression)

Design:
- Inner (repeat) split: even vs odd trials to avoid noise leakage.
- Neural PCA trained ONLY on train stimuli (using train repeats), then applied to test stimuli.
- Outer (stimulus) K-fold CV to test generalization to unseen images.
- Metrics on the test fold: R2 per PC, mean R2, Pearson r, and noise-normalized R2 (via split-half SB reliability on the test images).
- Optional permutation null over train pairings to get p-values for mean R2.

Author: Maria + Pläku 🐾
"""

import numpy as np
import pickle
from typing import List, Tuple
from sklearn.decomposition import PCA
from sklearn.linear_model import RidgeCV, Ridge
from sklearn.model_selection import KFold
from scipy.stats import pearsonr
from scipy.special import softmax
from skbio.stats.composition import clr

# -----------------------------
# CONFIG
# -----------------------------
VIT_PATH    = '/home/maria/Documents/HuggingMouseData/MouseViTEmbeddings/google_vit-base-patch16-224_embeddings_logits.pkl'
NEURAL_PATH = '/home/maria/LuckyMouse/pixel_transformer_neuro/data/processed/hybrid_neural_responses.npy'
AREAS_PATH  = '/home/maria/MITNeuralComputation/visualization/brain_area.npy'

AREAS       = ["VISp", "VISl", "VISrl", "VISal", "VISam", "VISpm"]
N_IMAGES, N_TRIALS = 118, 50
VAR_CUTOFF_VIT   = 0.90     # ViT PCA variance cutoff
VAR_CUTOFF_BRAIN = 0.90     # Brain PCA variance cutoff (fit on train stimuli only)
ALPHAS = np.logspace(-4, 3, 20)  # Ridge CV grid
K_OUTER = 5                        # K-fold across stimuli
N_PERM  = 200                      # Permutation null repetitions (set 0 to disable)
RANDOM_SEED = 42
rng = np.random.default_rng(RANDOM_SEED)

# -----------------------------
# Helpers
# -----------------------------
def fit_vit_pca(vit_logits: np.ndarray, var_cutoff=0.90, seed=RANDOM_SEED):
    Xv = softmax(np.asarray(vit_logits), axis=1)
    Xv = clr(Xv + 1e-12)
    pca_full = PCA(random_state=seed).fit(Xv)
    ncomp = int(np.searchsorted(np.cumsum(pca_full.explained_variance_ratio_), var_cutoff) + 1)
    pca = PCA(n_components=ncomp, random_state=seed)
    Zv = pca.fit_transform(Xv)
    # (optional) standardize PCs improves ridge conditioning
    Zv = (Zv - Zv.mean(axis=0)) / (Zv.std(axis=0) + 1e-8)
    return Zv, pca, ncomp

def split_repeats(dat_area: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
    """Return (X_train, X_test) as (neurons × images) by averaging even/odd repeats."""
    idx = np.arange(N_TRIALS)
    tr = idx[::2]  # even
    te = idx[1::2] # odd
    X_train = dat_area[:, :, tr, :].mean(axis=(2,3))
    X_test  = dat_area[:, :, te, :].mean(axis=(2,3))
    return X_train, X_test

def project_brain_pcs(X_train, X_test, img_idx_train, var_cutoff=0.90, seed=RANDOM_SEED):
    """
    Fit PCA on neural TRAIN repeats restricted to TRAIN IMAGES.
    Return Zb_train (all images in train basis), Zb_test (all images in train basis), and the PCA object.
    """
    # Fit on train-stimuli subset
    pca_full = PCA(random_state=seed).fit(X_train[:, img_idx_train].T)  # (n_train_imgs × n_neurons)
    ncomp = int(np.searchsorted(np.cumsum(pca_full.explained_variance_ratio_), var_cutoff) + 1)
    pca = PCA(n_components=ncomp, random_state=seed)
    # Fit on train images, but compute mean/components for the full projection
    Zb_train_imgs = pca.fit_transform(X_train[:, img_idx_train].T)  # (n_train_imgs × ncomp)
    # Now project ALL images into that basis (center by pca.mean_ in neuron space)
    Zb_train_all = (X_train.T - pca.mean_) @ pca.components_.T        # (N_IMAGES × ncomp)
    Zb_test_all  = (X_test.T  - pca.mean_) @ pca.components_.T
    # Center within image-space for regression stability
    Zb_train_all -= Zb_train_all.mean(axis=0, keepdims=True)
    Zb_test_all  -= Zb_test_all.mean(axis=0,  keepdims=True)
    return Zb_train_all, Zb_test_all, pca, ncomp

def r2_score_cols(Y_true, Y_pred):
    # R^2 per column
    ss_res = np.sum((Y_true - Y_pred)**2, axis=0)
    ss_tot = np.sum((Y_true - Y_true.mean(axis=0))**2, axis=0) + 1e-12
    return 1.0 - ss_res / ss_tot

def pearson_cols(Y_true, Y_pred):
    r = []
    for j in range(Y_true.shape[1]):
        rj = pearsonr(Y_true[:, j], Y_pred[:, j])[0]
        r.append(0.0 if np.isnan(rj) else rj)
    return np.array(r)

def split_half_ceiling(Zb_train_all, Zb_test_all, img_idx_test):
    """
    Compute split-half reliability (Spearman–Brown) on TEST images,
    using train/test repeat splits projected in the SAME (train) PCA basis.
    """
    # Restrict to held-out images
    A = Zb_train_all[img_idx_test]  # test images from even repeats
    B = Zb_test_all[img_idx_test]   # test images from odd repeats
    # Pearson corr per PC across test images
    A = A - A.mean(0); B = B - B.mean(0)
    num = np.sum(A * B, axis=0)
    den = np.sqrt(np.sum(A*A, axis=0) * np.sum(B*B, axis=0)) + 1e-12
    rho = np.clip(num/den, -1, 1)
    ceiling = (2 * rho) / (1 + rho + 1e-12)  # Spearman–Brown
    return ceiling

# -----------------------------
# Load data
# -----------------------------
with open(VIT_PATH, 'rb') as f:
    vit_logits = pickle.load(f)['natural_scenes']
Zv, vit_pca, vit_ncomp = fit_vit_pca(vit_logits, var_cutoff=VAR_CUTOFF_VIT, seed=RANDOM_SEED)
print(f"ViT PCs covering {int(VAR_CUTOFF_VIT*100)}% variance: {vit_ncomp}")

dat   = np.load(NEURAL_PATH, mmap_mode='r')                # (neurons × (images*trials*time))
areas = np.load(AREAS_PATH, allow_pickle=True)

# -----------------------------
# Outer stimulus K-fold CV (per area)
# -----------------------------
for AREA in AREAS:
    print(f"\n=== Area: {AREA} ===")

    mask = (areas == AREA)
    dat_area = dat[mask]
    if dat_area.size == 0:
        print("No data, skipping.")
        continue

    # reshape to (neurons × images × trials × time)
    n_neurons, n_total = dat_area.shape
    n_time = n_total // (N_IMAGES * N_TRIALS)
    dat_area = dat_area.reshape(n_neurons, N_IMAGES, N_TRIALS, n_time)

    # repeat split averages
    X_train_rep, X_test_rep = split_repeats(dat_area)  # (neurons × images), even/odd means

    # stimulus indices for outer CV
    stim_idx = np.arange(N_IMAGES)
    kf = KFold(n_splits=K_OUTER, shuffle=True, random_state=RANDOM_SEED)

    fold_metrics = []
    perm_pvals = []  # will fill if N_PERM > 0

    for fold, (img_tr, img_te) in enumerate(kf.split(stim_idx), 1):
        img_tr = stim_idx[img_tr]
        img_te = stim_idx[img_te]

        # Brain PCA: fit on TRAIN images using TRAIN repeats; project both repeats for ALL images
        Zb_train_all, Zb_test_all, brain_pca, brain_ncomp = project_brain_pcs(
            X_train_rep, X_test_rep, img_idx_train=img_tr, var_cutoff=VAR_CUTOFF_BRAIN, seed=RANDOM_SEED
        )

        # Features and targets per fold (restricted to the split)
        Xtr = Zv[img_tr]           # ViT PCs (train stimuli)
        Xte = Zv[img_te]           # ViT PCs (test stimuli)
        Ytr = Zb_train_all[img_tr] # neural PCs from train repeats (train images only)
        Yte_true = Zb_test_all[img_te]  # neural PCs from test repeats (test images only)

        # Fit ridge (multi-output)
        ridge = RidgeCV(alphas=ALPHAS, fit_intercept=False)
        ridge.fit(Xtr, Ytr)
        Yte_pred = ridge.predict(Xte)

        # Metrics on held-out stimuli
        R2_cols = r2_score_cols(Yte_true, Yte_pred)      # per PC
        R2_mean = float(np.mean(R2_cols))
        r_cols  = pearson_cols(Yte_true, Yte_pred)
        r_mean  = float(np.mean(r_cols))

        # Noise ceiling on held-out stimuli (SB)
        ceiling = split_half_ceiling(Zb_train_all, Zb_test_all, img_idx_test=img_te)
        R2_norm_cols = np.clip(R2_cols / np.maximum(ceiling, 1e-6), 0, 1)
        R2_norm_mean = float(np.mean(R2_norm_cols))

        print(f"[Fold {fold}] brainPCs={brain_ncomp:2d}  alpha={ridge.alpha_:g}  "
              f"R2_mean={R2_mean:.3f}  r_mean={r_mean:.3f}  R2_norm_mean={R2_norm_mean:.3f}")

        fold_metrics.append({
            'fold': fold,
            'alpha': float(ridge.alpha_),
            'brain_ncomp': int(brain_ncomp),
            'R2_mean': R2_mean,
            'r_mean': r_mean,
            'R2_norm_mean': R2_norm_mean,
            'R2_cols': R2_cols,
            'r_cols': r_cols,
            'ceiling': ceiling,
        })

        # Permutation null (optional): shuffle pairing on train, evaluate on same test split
        if N_PERM > 0:
            null_scores = []
            for s in range(N_PERM):
                perm = rng.permutation(len(img_tr))
                ridge_null = Ridge(alpha=ridge.alpha_, fit_intercept=False)
                ridge_null.fit(Xtr, Ytr[perm])     # break stimulus alignment in TRAIN
                Yte_null = ridge_null.predict(Xte)
                null_scores.append(float(np.mean(r2_score_cols(Yte_true, Yte_null))))
            null_scores = np.array(null_scores)
            p = (1 + np.sum(null_scores >= R2_mean)) / (1 + len(null_scores))
            print(f"         perm p≈{p:.4f} (vs mean R2)")
            perm_pvals.append(p)

    # Aggregate across folds
    R2_mean_over_folds      = np.mean([m['R2_mean'] for m in fold_metrics])
    R2_norm_mean_over_folds = np.mean([m['R2_norm_mean'] for m in fold_metrics])
    r_mean_over_folds       = np.mean([m['r_mean'] for m in fold_metrics])
    print(f"\n[Area {AREA}]  "
          f"Stimulus-CV results:  R2_mean={R2_mean_over_folds:.3f}  "
          f"R2_norm_mean={R2_norm_mean_over_folds:.3f}  r_mean={r_mean_over_folds:.3f}")

    if N_PERM > 0 and len(perm_pvals) > 0:
        # Combine fold-wise p-values with Fisher's method (optional)
        from math import log
        from scipy.stats import chi2
        stat = -2 * np.sum([log(max(p, 1e-12)) for p in perm_pvals])
        df = 2 * len(perm_pvals)
        p_comb = 1 - chi2.cdf(stat, df)
        print(f"[Area {AREA}]  Combined permutation p (Fisher): {p_comb:.4e}")


ViT PCs covering 90% variance: 44

=== Area: VISp ===
[Fold 1] brainPCs=73  alpha=1000  R2_mean=-0.066  r_mean=0.038  R2_norm_mean=0.014
         perm p≈0.0348 (vs mean R2)
[Fold 2] brainPCs=74  alpha=1000  R2_mean=-0.066  r_mean=0.033  R2_norm_mean=0.010
         perm p≈0.2189 (vs mean R2)
[Fold 3] brainPCs=73  alpha=1000  R2_mean=-0.047  r_mean=0.107  R2_norm_mean=0.018
         perm p≈0.0050 (vs mean R2)
[Fold 4] brainPCs=74  alpha=1000  R2_mean=-0.050  r_mean=0.040  R2_norm_mean=0.011
         perm p≈0.1443 (vs mean R2)
[Fold 5] brainPCs=74  alpha=1000  R2_mean=-0.078  r_mean=0.031  R2_norm_mean=0.011
         perm p≈0.0846 (vs mean R2)

[Area VISp]  Stimulus-CV results:  R2_mean=-0.061  R2_norm_mean=0.013  r_mean=0.050
[Area VISp]  Combined permutation p (Fisher): 1.1687e-03

=== Area: VISl ===
[Fold 1] brainPCs=71  alpha=428.133  R2_mean=-0.150  r_mean=0.002  R2_norm_mean=0.017
         perm p≈0.5323 (vs mean R2)
[Fold 2] brainPCs=72  alpha=1000  R2_mean=-0.036  r_mean=0.086  R2_